# Module 불러오기

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from gensim.models import Word2Vec
from scipy.stats import mode
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
import warnings
import gensim
import pickle
from scipy.io import mmread
warnings.filterwarnings(action='ignore')

In [108]:
import pandas as pd
import warnings
import re
from konlpy.tag import Okt
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import numpy as np
warnings.filterwarnings(action='ignore')

# 데이터 불러오기

In [52]:
df = pd.read_csv('./data/beer_only_eng.csv', index_col = 0)
df

,ID,beer_name,date,grade,Aroma,Appearance,Flavor,Mouthfeel,Overall,beer_review,lang
0,"italianjohn(5,276)",Kloud Original Gravity,"September 6, 2021",2.8,5.0,3.0,5.0,3.0,12.0,Clear golden blonde pour with a thin white hea...,en
1,orda(284),Kloud Original Gravity,"August 29, 2021",3.9,10.0,5.0,10.0,5.0,9.0,Appearance: golden amber; slight head\nTaste: ...,en
2,"Radek Kliber(8,065)",Kloud Original Gravity,"June 5, 2021",2.4,6.0,3.0,4.0,2.0,9.0,"Bottle 330 ml \n\nAroma: nice, fresh, on dry s...",en
3,"geologyguy(2,575)",Kloud Original Gravity,"May 26, 2021",2.3,4.0,3.0,4.0,3.0,9.0,Smell is light of malt...taste is a that malt ...,en
4,Wolff90(203),Kloud Original Gravity,"February 16, 2021",1.0,1.0,1.0,1.0,1.0,6.0,Rating copied from Untappd. Had while flying A...,en
...,...,...,...,...,...,...,...,...,...,...,...
83186,bourne(161),Cafri,"March 21, 2006",1.3,2.0,2.0,3.0,2.0,4.0,I didnt think this was beer when I bought it. ...,en
83187,curvedbeer(11),Cafri,"November 6, 2005",1.0,2.0,3.0,1.0,1.0,3.0,Of all of the commercially brewed beers I’ve t...,en
83188,"tomciccateri(1,033)",Cafri,"September 5, 2007",2.2,3.0,3.0,4.0,2.0,10.0,"From a can onboard a KAL flight. 4.2%, with no...",en
83189,"Papsoe(25,039)",Cafri,"June 13, 2005",2.1,5.0,2.0,4.0,2.0,8.0,(Bottle 48 cl) Again - for Korea - this beauti...,en


In [53]:
df.drop_duplicates(inplace=True) # 중복값제거

In [54]:
df

,ID,beer_name,date,grade,Aroma,Appearance,Flavor,Mouthfeel,Overall,beer_review,lang
0,"italianjohn(5,276)",Kloud Original Gravity,"September 6, 2021",2.8,5.0,3.0,5.0,3.0,12.0,Clear golden blonde pour with a thin white hea...,en
1,orda(284),Kloud Original Gravity,"August 29, 2021",3.9,10.0,5.0,10.0,5.0,9.0,Appearance: golden amber; slight head\nTaste: ...,en
2,"Radek Kliber(8,065)",Kloud Original Gravity,"June 5, 2021",2.4,6.0,3.0,4.0,2.0,9.0,"Bottle 330 ml \n\nAroma: nice, fresh, on dry s...",en
3,"geologyguy(2,575)",Kloud Original Gravity,"May 26, 2021",2.3,4.0,3.0,4.0,3.0,9.0,Smell is light of malt...taste is a that malt ...,en
4,Wolff90(203),Kloud Original Gravity,"February 16, 2021",1.0,1.0,1.0,1.0,1.0,6.0,Rating copied from Untappd. Had while flying A...,en
...,...,...,...,...,...,...,...,...,...,...,...
83186,bourne(161),Cafri,"March 21, 2006",1.3,2.0,2.0,3.0,2.0,4.0,I didnt think this was beer when I bought it. ...,en
83187,curvedbeer(11),Cafri,"November 6, 2005",1.0,2.0,3.0,1.0,1.0,3.0,Of all of the commercially brewed beers I’ve t...,en
83188,"tomciccateri(1,033)",Cafri,"September 5, 2007",2.2,3.0,3.0,4.0,2.0,10.0,"From a can onboard a KAL flight. 4.2%, with no...",en
83189,"Papsoe(25,039)",Cafri,"June 13, 2005",2.1,5.0,2.0,4.0,2.0,8.0,(Bottle 48 cl) Again - for Korea - this beauti...,en


## 데이터 맥주 별로 합치기

In [55]:
df2 = df[['beer_name','beer_review']]

In [56]:
df2

,beer_name,beer_review
0,Kloud Original Gravity,Clear golden blonde pour with a thin white hea...
1,Kloud Original Gravity,Appearance: golden amber; slight head\nTaste: ...
2,Kloud Original Gravity,"Bottle 330 ml \n\nAroma: nice, fresh, on dry s..."
3,Kloud Original Gravity,Smell is light of malt...taste is a that malt ...
4,Kloud Original Gravity,Rating copied from Untappd. Had while flying A...
...,...,...
83186,Cafri,I didnt think this was beer when I bought it. ...
83187,Cafri,Of all of the commercially brewed beers I’ve t...
83188,Cafri,"From a can onboard a KAL flight. 4.2%, with no..."
83189,Cafri,(Bottle 48 cl) Again - for Korea - this beauti...


In [57]:
df_sum = pd.DataFrame(df2.groupby('beer_name').sum())
df_sum

,beer_review
beer_name,
7Bräu Gompyo Wheatbeer,(Can) Pours clear pale golden yellow with loos...
ARK Seoulite Ale,Darker amber with smooth yet aggressive creamy...
Asahi Super Dry,Golden with quickly disintegrating white head....
Asahi Super Dry Black,"Dark copper red beer , with a big tan coloured..."
Bavaria 8.6 (Original),"Can. Clear golden, white head. Candy, sugary a..."
...,...
Tsingtao,500ml can. Pours a clear pale gold with a thin...
Tsingtao Wheat Beer,"Intense banana and clove aromas, pale yellow w..."
Victoria Bitter,Works very well above 32 degrees Celcius and o...


In [1]:
df_sum.columns = [['beer_name', 'beer_review']]

NameError: name 'df_sum' is not defined

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82104\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\82104\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
# 불용어 제거
stops = set(stopwords.words('english'))
# no_stops = [word for word in no_capitals if not word in stops]

# stemmer = nltk.stem.SnowballStemmer('english')
# stemmer_words = [stemmer.stem(word) for word in no_stops]

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [69]:
df_sum.beer_review.iloc[0]

'(Can) Pours clear pale golden yellow with loose short lived white head. Unusual aroma of peach flavouring. Quite pleasant but odd. Light body with bright carbonation. Taste is an interesting balance of the peach flavour and classic wheat beer. The peach is dominant at the start but gives way to the yeast and wheat flavours. There’s some modest bitterness in the finish. Unusual and enjoyable.'

In [74]:
df_sum.beer_review.iloc[0]

'(Can) Pours clear pale golden yellow with loose short lived white head. Unusual aroma of peach flavouring. Quite pleasant but odd. Light body with bright carbonation. Taste is an interesting balance of the peach flavour and classic wheat beer. The peach is dominant at the start but gives way to the yeast and wheat flavours. There’s some modest bitterness in the finish. Unusual and enjoyable.'

# 명사 동사 형용사만 남기기

In [135]:
# nltk = nltk.word_tokenize() #tokenizer 로드
TAG_NORMAL = {'JJR', 'JJS', 'NN', 'NNS', 'NNP','NNP','NNPS','RB','RBR','RBS','VBD','VB','VBN','VBG'}
df_fin = pd.DataFrame()

cleaned_sentences = []
count = 0
token_stop = []

for sentence in tqdm(df_sum.beer_review):
    sentences = re.sub(r"[^a-zA-Z0-9]"," ", sentence) # 한글이 아닌 문자는 공백으로 표시
    sentences = sentences.lower()
    new_string = sentences.replace('  ',' ')
    token = nltk.word_tokenize(new_string) # 토크화
    token_stop = [word for word in token if not word in stops]
    token_stop = nltk.pos_tag(token_stop)
    df_token = pd.DataFrame(token_stop, columns=['word', 'class'])
    df_cleaned_token = df_token[df_token['class'].isin(TAG_NORMAL)]['word']  # TAG_NORMAL 에 있는 품사만 뽑아내는 작업
    cleaned_sentence: str = ' '.join(df_cleaned_token)  # 단어들 이어붙이기
    df_clean = pd.DataFrame([cleaned_sentence], columns = ['review'])
    df_fin = pd.concat([df_fin, df_clean])

100%|██████████████████████████████████████████████████████████████████████████████████| 87/87 [01:29<00:00,  1.02s/it]


In [137]:
df_fin.reset_index(drop = True, inplace = True)
df_fin

,review
0,pours yellow loose lived head aroma peach flav...
1,darker amber yet creamy foam creamy foam caram...
2,quickly disintegrating head sweet malty notes ...
3,copper beer tan coloured head toasty malt arom...
4,head candy aroma corn candy alcohol bought mos...
...,...
82,pours gold fizzy lived head lacing aroma malt ...
83,banana clove aromas pale hint turbidities swee...
84,works well degrees neutrals world coloured fai...
85,pale yellow colour dense head lasts minutes cl...


In [ ]:
df_sum['']

In [140]:
df_fin['beer'] = df_sum.index.to_list()

In [142]:
df_fin = df_fin[['beer','review']]
df_fin

,beer,review
0,7Bräu Gompyo Wheatbeer,pours yellow loose lived head aroma peach flav...
1,ARK Seoulite Ale,darker amber yet creamy foam creamy foam caram...
2,Asahi Super Dry,quickly disintegrating head sweet malty notes ...
3,Asahi Super Dry Black,copper beer tan coloured head toasty malt arom...
4,Bavaria 8.6 (Original),head candy aroma corn candy alcohol bought mos...
...,...,...
82,Tsingtao,pours gold fizzy lived head lacing aroma malt ...
83,Tsingtao Wheat Beer,banana clove aromas pale hint turbidities swee...
84,Victoria Bitter,works well degrees neutrals world coloured fai...
85,Volfas Engelman Grünberger Hefeweizen,pale yellow colour dense head lasts minutes cl...


In [143]:
df_fin.to_csv('./data/beer_CBF.csv')

In [145]:
abc = pd.read_csv('./data/beer_CBF.csv', index_col = 0)
abc

,beer,review
0,7Bräu Gompyo Wheatbeer,pours yellow loose lived head aroma peach flav...
1,ARK Seoulite Ale,darker amber yet creamy foam creamy foam caram...
2,Asahi Super Dry,quickly disintegrating head sweet malty notes ...
3,Asahi Super Dry Black,copper beer tan coloured head toasty malt arom...
4,Bavaria 8.6 (Original),head candy aroma corn candy alcohol bought mos...
...,...,...
82,Tsingtao,pours gold fizzy lived head lacing aroma malt ...
83,Tsingtao Wheat Beer,banana clove aromas pale hint turbidities swee...
84,Victoria Bitter,works well degrees neutrals world coloured fai...
85,Volfas Engelman Grünberger Hefeweizen,pale yellow colour dense head lasts minutes cl...


In [ ]:
df['cleaned_review'] = cleaned_sentences
print(df.head())
df = df[['title', 'cleaned_review']]

In [49]:
df_cleaned_token

,word,class
0,probably,RB
1,favorite,JJ
2,korean,JJ
3,beer,NN
4,available,JJ
5,ob,JJ
6,hite,NN
7,pours,NNS
9,yellow,JJ
10,thick,NN


In [50]:
df_token

,word,class
0,probably,RB
1,favorite,JJ
2,korean,JJ
3,beer,NN
4,available,JJ
5,ob,JJ
6,hite,NN
7,pours,NNS
8,straw,VBP
9,yellow,JJ


In [9]:
df['beer_review'] = re.sub(r"[a-zA-Z0-9_]", "", df.beer_review) 
df

TypeError: expected string or bytes-like object

In [ ]:
# 데이터 불러오기
df_review_one_sentence = pd.read_csv(
    "./data/beer_only_eng.csv", index_col=0)
df_review_one_sentence.reset_index(inplace = True, drop = True)

In [ ]:
def getRecommendation(cosine_sim):
    simScores = list(enumerate(cosine_sim[-1])) 
    # 가장 높은 유사도 추출
    simScores = sorted(simScores, key=lambda x:x[1], 
                       reverse=True)
    
    # 가장 높은 유사도 40개 추출
    simScores = simScores[1:40]
    movieidx = [i[0] for i in simScores]
    RecMovielist = df_review_one_sentence.iloc[movieidx].reset_index(drop = True)
    
    # topic_mode = 가장 높은 유사도의 lda_topic
    topic_mode = mode(RecMovielist['lda_topic'])[0]
    res = []
    for i in movieidx:
        # 키워드와의 높은 유사도를 가진 값들 중 lda topic값이 같은 것들만 추출(10개)
        if df_review_one_sentence['lda_topic'].loc[i] == topic_mode[0]:
            res.append(i)
            if len(res) ==10:
                break
    RecMovielist = df_review_one_sentence.iloc[res].reset_index(drop = True)
    print(RecMovielist)
    
    # subplot으로 추출된 영화 포스터 visualization
    fig = plt.figure(figsize=(20,20))
    for index, row in RecMovielist.iterrows():
        response = requests.get(row['image_link'])
        img = Image.open(BytesIO(response.content))
        fig.add_subplot(2, 5, index + 1)
        plt.imshow(img)
        plt.title(row['title'])
    return df_review_one_sentence['title'][movieidx]

# 이전에 학습한 embedding model 불러오기
embedding_model = Word2Vec.load("C:/Users/82104/Documents/Sessac/자연어처리/가상환경/1st_project/model/final/word2VecModel_final.model")

# 키워드 입력
key_word = '스포츠'
sentence = [key_word] * 10

# 유사도가 높은 단어 순으로 가중치(keyword의 갯수)를 사용해 keyword와 유사도 높은 영화 찾기
if key_word in embedding_model.wv.index_to_key:
    sim_word = embedding_model.wv.most_similar(key_word, topn=10)
    print(sim_word)
    labels = []
    for label, _ in sim_word:
        labels.append(label)
    print(labels)
    for i, word in enumerate(labels):
        sentence += [word] * (9-i)
sentence = ' '.join(sentence)
print(sentence)

# 이전에 학습한 TFIDF 모델 호출
with open("C:/Users/82104/Documents/Sessac/자연어처리/가상환경/1st_project/model/final/tfidf.pickle", "rb") as fr:
    Tfidf = pickle.load(fr)
Tfidf_matrix = mmread("C:/Users/82104/Documents/Sessac/자연어처리/가상환경/1st_project/model/final/tfidf_movie_review.mtx")

# TFIDF 모델 transform
sentence_vec = Tfidf.transform([sentence])

# 코사인 유사도 계산
cosine_sim = linear_kernel(sentence_vec,
                           Tfidf_matrix)
recommendation = getRecommendation(cosine_sim)
print(recommendation)

### 쓰레기통

In [ ]:
df_cleaned_token = df_token[(df_token['class'] == 'JJ') | # 명사, 동사, 형용사로 이루어진 df 만들기
                            (df_token['class'] == 'JJR') |
                            (df_token['class'] == 'JJS') |
                            (df_token['class'] == 'NN') |
                           (df_token['class'] == 'NNS') |
                           (df_token['class'] == 'NNP') |
                           (df_token['class'] == 'NNPS') |
                           (df_token['class'] == 'RB') |
                            (df_token['class'] == 'RBR') |
                            (df_token['class'] == 'RBS') |
                            (df_token['class'] == 'VB') |
                            (df_token['class'] == 'VBD') |
                            (df_token['class'] == 'VBN') |
                           (df_token['class'] == 'VBG')]